# Tutorials
**Note**: The examples are found in the following jupyter notebook `xmen/examples/examples.ipynb` where they can be run interactively. Before running the notebook make sure that `xmen` is on your `PYTHONPATH` following the steps given in Setup.

## Back to Basics

### Defining Experiments in Python

Experiments at there most basic are typically defined by some function code which operates on a set of parameters. We typically might define an experiment in python by defining a function which takes in a set of arguments and does something with them. For example a simple experiment script might look something like:

In [1]:
#<hello_world.py>
from argparse import ArgumentParser

parser = ArgumentParser()
parser.add_argument('-a', type=str, default='Hello', help="the first argument")
parser.add_argument('-b', type=str, default='World!', help="the second argument")

def hello_world(
    a: str = 'Hello', 
    b: str = 'World'
):
    """A simple python experiment.
    
    a (str): the first argument
    b (str): the second argument
    """
    print(f'{a}  {b}')

    ...  #  Whatever other experiment code you want

Having defined an experiment we can now run it from the command line as:

```bash
python hello_world.py -a Hello -b World!
```

This is fine for simple experiments with only a few parameters, however, once we start scaling up defining experiments in this way becomes increasingly cumbersome. Having to define the parameter names, type, defaults and help, once in the argument parser, once in the function signature and also in the functions doc string is laborious and increases the chance of errors! 

### Defining Experiments in Xmen

Xmen side-steps the need to redefine parameters; a functional experiment corresponds to a single function definiton:

In [2]:
from xmen.experiment import Root

def hello_world(
    root: Root,
    a: str = 'Hello',  # the first argument
    b: str = 'World'   # the second argument
):
    """A simple experiment using the xmen functional api"""
    print(f'{a}  {b}')

In comparison to the previous example, parameters are defined once in the function signature, allowing the name, type, default and help message of the parameter to be defined in a single line of code. 

The only change required to the function ``hello_world`` is the introduction of a new first argument `root`. But what is this for? Before being run each experiment is assigned a particular directory (in this case `/tmp/hello_world`) which is passed to the experiment in `root`. Whilst, this directory is unused in our example, principally there are two things that an experiment might do with ``root``:
1. ``root.directory`` gives the directory assigned to the experiment which the experiment is free (and encouraged!) to use as it wishes
2. ``root.message(...)`` provides a simple messaging protocol with root directory

For example we could modify the previouse experiment to use root as:

In [3]:
from xmen.experiment import Root

def hello_world(
    root: Root,        # experiments are assigned a root before being executed
    a: str = 'Hello',  # the first argument
    b: str = 'World'   # the second argument
):
    """A hello world experiment designed to demonstrate
    defining experiments through the functional experiment api"""
    print(f'{a}  {b}')

    ...  #  Whatever other experiment code you want

    with open(root.directory + '/out.txt', 'w') as f:
        f.write(f'{a} {b}')
    root.message({'a': a, 'b': b})

### Xmen takes care of the rest...

#### Command Line Interface

Alongisde a python api xmen also provides a command line interfce for running experiments:

In [11]:
%%bash
xmen

usage: xman [-h] [--list] [--add MODULE NAME MODULE NAME] [--remove REMOVE]
            [name [name ...]] ...

||||||||||||||||||||||||| WELCOME TO ||||||||||||||||||||||||||
||                                                           ||
||    \\\  ///  |||\\        //|||  |||||||||  |||\\   |||   ||
||     \\\///   |||\\\      ///|||  |||        |||\\\  |||   ||
||      ||||    ||| \\\    /// |||  ||||||     ||| \\\ |||   ||
||     ///\\\   |||  \\\  ///  |||  |||        |||  \\\|||   ||
||    ///  \\\  |||   \\\///   |||  |||||||||  |||   \\|||   ||
||                                                           ||
|||||||||||| FAST - REPRODUCIBLE - EXPERIMENTATION ||||||||||||

positional arguments:
  name                  The name of the experiment to run
  flags                 Python flags (pass --help for more info)

optional arguments:
  -h, --help            show this help message and exit
  --list, -l            List available python experiments
  --add MODULE NAME MODULE NAME


#### Running an Experiment

Every experiment conforming to the xmen python api can be run using the xmen command line interface. The function `hello_world` above is actually a copy of `hello_world` in `xmen.examples.hello_world`. It can be run from the command line as

In [5]:
%%bash
xmen --add xmen.examples.hello_world hello_world
xmen hello_world -x /tmp/hello_world

Hello  World


In the first step, xmen retrieves the ``hello_world`` function from the `hello_world.py` module and adds it to its internal database. And in the second, `xmen` runs the experiment linking it to the directory `/tmp/test`.

#### Automatic Argument Parser

Xmen automatically allows parameters to be set from the command line without defining a specific argument parser

In [6]:
%%bash
xmen hello_world -u "{a: Bye Bye, b: Planet!}" -x /tmp/hello_world

Updating parameters {'a': 'Bye Bye', 'b': 'Planet!'}
Bye Bye  Planet!


#### Help Generation
Alongside running experiments the command line tool automatically takes care of documentation for you.

In [7]:
%%bash
xmen hello_world

A hello world experiment designed to demonstrate
    defining experiments through the functional experiment api

Parameters:
    a: str=Hello ~ the first argument
    b: str=World ~ the second argument

For more help use --help.


#### Book Keeping, Vesioning and Reproducibility
The other advantage of linking an experiment with a root is book keeping. Every singe time an experiment is run xmen automatically logs all the information that might be needed to repruce the experiment at a later date. For example looking at

In [10]:
%%bash
cat /tmp/hello_world/params.yml

_created: 11-14-20-15:29:46  # _created: str=now_time ~ The date the experiment was created
_messages: # _messages: Dict[Any, Any]={} ~ Messages left by the experiment
  a: Hello
  b: World
_meta: # _meta: Optional[Dict]=None ~ The global configuration for the experiment manager
  mac: '0x6c96cfdb71b9'
  host: robw-MacBook-2
  user: robweston
  home: /Users/robweston
_name: hello_world # _name: Optional[str]=None ~ The name of the experiment (under root)
_purpose: '' # _purpose: Optional[str]=None ~ A description of the experiment purpose
_root: /tmp # _root: Optional[str]=None ~ The root directory of the experiment
_status: finished # _status: str='default' ~ One of ['default' | 'created' | 'running' | 'error' | 'finished']
_version: # _version: Optional[Dict[Any, Any]]=None ~ Experiment version information. See `get_version`
  module: xmen.examples.hello_world
  function: hello_world
  path: /Users/robweston/miniconda3/envs/python3.6/lib/python3.6/site-packages/xmen/examples/hello_wo

we can see:
    - `_created` time the experiment was run
    - `_messages` any messages logged with the experiment during execution
    - `_meta` information about the system the experiment was run on
    - `_status` the state of the experiment. If an error occurs during excution then the experiment state will be updated to `'errror'`.
    - `_version` the information needed to re-run the experiment. If the file lives in a git repo then the git commit, path to the repositoy and remote repository are also loggeed automatically.

